# Overview

This notebook does the initial processing and areal interpolation of census data for each of the three service areas around the San Gabriel National Monument.

# Setup

In [97]:
# Import libraries
import arcpy, pandas as pd, numpy as np, arcgis, os
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [98]:
# Set workspace settings
arcpy.env.workspace  = r"C:\Users\kathr\Documents\outdoor-alliance\san-gabriel\san-gabriel-analysis\sg_output.gdb"
arcpy.env.overwriteOutput = True

In [99]:
# Set gdb path
gdb_path = r"C:\Users\kathr\Documents\outdoor-alliance\san-gabriel\san-gabriel-analysis\sg_output.gdb"

In [100]:
# Set path to folder where all data files are stored
data_folder = r"C:\Users\kathr\Documents\outdoor-alliance\san-gabriel\data"

# Define functions for areal interpolation

In [101]:
# census_df (pd.DataFrame) : dataframe for the service area with census information
# int_df (pd.DataFrame) : calculated intersection for the service area
# returns updated census_df (pd.DataFrame)

def interpolate_racial(census_df, int_df):
    # By default, the datatype of most columns in the census DF is "object" - need to convert to types like int/float/string for the multiplication
    census_df = census_df.convert_dtypes()
    
    # Get column names of numeric data
    num_cols = census_df.select_dtypes([np.number]).columns
    
    # Remove OBJECTID from the list of columns
    num_cols = np.delete(num_cols, np.where(num_cols == "OBJECTID"))
    
    # Sort both dataframes by GEOID so they will align
    census_df = census_df.sort_values(by = ["GEOID"]).reset_index(drop = True)
    int_df = int_df.sort_values(by = ["GEOID"]).reset_index(drop = True)
    
    # Multiply each numeric column by the percent of the census tract that is in the service area and save the result back into the same column
    for col in num_cols:
        census_df[col] = census_df[col] * (int_df["PERCENTAGE"] / 100)
        
    return census_df

In [102]:
# census_df (pd.DataFrame) : dataframe for the service area with census information
# int_df (pd.DataFrame) : calculated intersection for the service area
# nhgis_edu (pd.DataFrame) : educational attainment data
# returns subset of updated education information (pd.DataFrame)

def interpolate_edu(census_df, int_df, nhgis_edu):
    # Filter to tracts in this service area
    edu_sub = nhgis_edu[nhgis_edu["GEOID"].str[-11:].isin(census_df["GEOID"])]

    # Reformat GEOID column to the last 11 digits instead of full code (to match existing data)
    edu_sub["GEOID"] = edu_sub["GEOID"].str[-11:]

    # Sort by GEOID
    edu_sub = edu_sub.sort_values(by = ["GEOID"]).reset_index(drop = True)
    
    # Subset to columns of interest
    edu_cols = ["GEOID", # tract ID
                "AMRZE001", # Total count
                "AMRZE022", # Bachelor's degree 
                "AMRZE023", # Master's degree 
                "AMRZE024", # Professional school degree 
                "AMRZE025"] # Doctorate degree 

    edu_sub = edu_sub[edu_cols]
    
    # Multiply each numeric column by the percent of the census tract that is in the service area and save the result back into the same column
    for col in edu_cols[1:]:
        edu_sub[col] = edu_sub[col] * (int_df["PERCENTAGE"] / 100)
        
    return edu_sub

In [103]:
# census_df (pd.DataFrame) : dataframe for the service area with census information
# int_df (pd.DataFrame) : calculated intersection for the service area
# nhgis_inc (pd.DataFrame) : income data
# returns subset of updated education information (pd.DataFrame)

def interpolate_inc(census_df, int_df, nhgis_inc):
    # Filter to tracts in this service area
    inc_sub = nhgis_inc[nhgis_inc["GEOID"].str[-11:].isin(census_df["GEOID"])]

    # Reformat GEOID column to the last 11 digits instead of full code (to match existing data)
    inc_sub["GEOID"] = inc_sub["GEOID"].str[-11:]

    # Sort by GEOID
    inc_sub = inc_sub.sort_values(by = ["GEOID"]).reset_index(drop = True)
    
    # Subset to columns of interest
    inc_cols = ["GEOID", # tract ID
                "AMZME001", # Total count
                "AMZME002", # Under .50
                "AMZME003"] # .50 - .99 

    inc_sub = inc_sub[inc_cols]
    
    # Multiply each numeric column by the percent of the census tract that is in the service area and save the result back into the same column
    for col in inc_cols[1:]:
        inc_sub[col] = inc_sub[col] * (int_df["PERCENTAGE"] / 100)
        
    return inc_sub

# State of CA

In [104]:
# Read in educational attainment data
nhgis_edu = pd.read_csv(os.path.join(data_folder, r"nhgis0001_shapefile_tl2020_us_tract_2020\nhgis0001_ds249_20205_tract.csv"), encoding = "latin1")

# Make a copy to modify
edu_df = nhgis_edu.copy()

In [105]:
# Read in income data 
nhgis_inc = pd.read_csv(os.path.join(data_folder, r"nhgis0001_shapefile_tl2020_us_tract_2020\nhgis0002_ds249_20205_tract.csv"), encoding = "latin1")

# Make a copy to modify
inc_df = nhgis_inc.copy()

In [106]:
# Read in census data for all of CA
ca_census = pd.DataFrame.spatial.from_featureclass("ca_census")

In [107]:
# Trim GEOID from NHGIS data to match other census data
edu_df["GEOID"] = edu_df["GEOID"].str[-11:]
inc_df["GEOID"] = inc_df["GEOID"].str[-11:]

In [108]:
# Select columns of interest from NHGIS data
edu_cols = ["GEOID", # tract ID
            "AMRZE001", # Total count
            "AMRZE022", # Bachelor's degree 
            "AMRZE023", # Master's degree 
            "AMRZE024", # Professional school degree 
            "AMRZE025"] # Doctorate degree 

edu_df = edu_df[edu_cols]
    
inc_cols = ["GEOID", # tract ID
            "AMZME001", # Total count
            "AMZME002", # Under .50
            "AMZME003"] # .50 - .99 

inc_df = inc_df[inc_cols]

In [109]:
# Cast GEOID to int to facilitate joining with other census data
ca_census = ca_census.astype({"GEOID": "int64"})
edu_df = edu_df.astype({"GEOID": "int64"})
inc_df = inc_df.astype({"GEOID": "int64"})

In [110]:
ca_census = ca_census.merge(edu_df, on = "GEOID", how = "left")
ca_census = ca_census.merge(inc_df, on = "GEOID", how = "left")

In [111]:
# Write combined df back to feature class
ca_census.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "combined_ca_census"), 
                                        sanitize_columns = False)

'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\combined_ca_census'

# Service Area with Rings

"Rings" means that the inner service areas are not included in the outer service areas. For example, the 5-30 mile service area does not include the 0-5 mile service area.

## Split network dataset

In [3]:
# Create pandas dataframe from the network dataset layer (rings)
network_df = pd.DataFrame.spatial.from_featureclass("nd_output_rings")

In [4]:
# View the dataframe
network_df

,ObjectID,Name,FromBreak,ToBreak,FacilityOID,FacilityID,SHAPE
0,1,30 - 90,30.0,90.0,None,None,"{""rings"": [[[-13089225.466699999, 4236206.6894..."
1,2,5 - 30,5.0,30.0,None,None,"{""rings"": [[[-13103400.466699999, 4128411.6894..."
2,3,0 - 5,0.0,5.0,None,None,"{""rings"": [[[-13138670.466699999, 4096601.6894..."


In [5]:
# Split service area output into 3 separate polygons based on service area
sa_1 = network_df[network_df["FromBreak"] == 0.0]
sa_2 = network_df[network_df["FromBreak"] == 5.0]
sa_3 = network_df[network_df["FromBreak"] == 30.0]

In [43]:
# Write the service areas back to feature classes
sa_1.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "service_area_1_rings"), 
                                        sanitize_columns = False)
sa_2.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "service_area_2_rings"), 
                                        sanitize_columns = False)
sa_3.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "service_area_3_rings"), 
                                        sanitize_columns = False)

'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\service_area_3_rings'

## Calculate intersection of tracts in each service area

In [ ]:
# Census
# Set variables
in_features = "census_clipped"
clip_features = "nd_output_rings"
out_feature_class = "nd_rings_census"

In [ ]:
# Run clip
arcpy.analysis.Clip(in_features, clip_features, out_feature_class)

In [ ]:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/analysis/tabulate-intersection.htm
arcpy.analysis.TabulateIntersection(in_zone_features = "nd_rings_census", 
                                    zone_fields = "GEOID", # tract identifier
                                    in_class_features = "nd_output_rings", 
                                    out_table = "rings_census_intersection", 
                                    class_fields = "Name" # service area name
                                    )

In [ ]:
# The output PERCENTAGE field records the percentage of the zone feature that is intersected by the class
# Zone feature = census tracts
# Class = service area
# Percent of the census tract that is intersected by a particular service area

## 0 - 5 mile service area

In [4]:
# Census tracts
# Set variables
in_features = "census_clipped"
clip_features = "service_area_1_rings"
out_feature_class = "sa_1_rings_census"

In [5]:
# Run clip
arcpy.analysis.Clip(in_features, clip_features, out_feature_class)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sa_1_rings_census'>

### Areal interpolation of census attributes

In [112]:
# Create pandas dataframe from census feature class
sa_1_census_df = pd.DataFrame.spatial.from_featureclass("sa_1_rings_census")

# Create pandas dataframe from intersection table
rings_census_int_df = pd.DataFrame.spatial.from_table("rings_census_intersection")

# Subset intersection table to only the service area of interest
sa_1_int = rings_census_int_df[rings_census_int_df["Name"] == "0 - 5"].reset_index(drop = True)

In [113]:
# Calculate racial attributes
sa_1_census_df = interpolate_racial(sa_1_census_df, sa_1_int)

In [114]:
# Calculate education attributes
sa_1_edu = interpolate_edu(sa_1_census_df, sa_1_int, nhgis_edu)

[102]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [115]:
# Calculate income attributes
sa_1_inc = interpolate_inc(sa_1_census_df, sa_1_int, nhgis_inc)

[103]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [116]:
# Cast GEOID to int to facilitate joining with other census data
sa_1_census_df = sa_1_census_df.astype({"GEOID": "int64"})
sa_1_edu = sa_1_edu.astype({"GEOID": "int64"})
sa_1_inc = sa_1_inc.astype({"GEOID": "int64"})

# Join edited columns to the service area dataframe
sa_1_census_df = sa_1_census_df.merge(sa_1_edu, on = "GEOID", how = "left")
sa_1_census_df = sa_1_census_df.merge(sa_1_inc, on = "GEOID", how = "left")

In [117]:
# Write combined df back to feature class
sa_1_census_df.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "combined_census_sa_1_rings"), 
                                        sanitize_columns = False)

'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\combined_census_sa_1_rings'

## 5 - 30 mile service area

In [6]:
# Census tracts
# Set variables
in_features = "census_clipped"
clip_features = "service_area_2_rings"
out_feature_class = "sa_2_rings_census"

In [7]:
# Run clip
arcpy.analysis.Clip(in_features, clip_features, out_feature_class)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sa_2_rings_census'>

### Areal interpolation of census attributes

In [118]:
# Create pandas dataframe from census feature class
sa_2_census_df = pd.DataFrame.spatial.from_featureclass("sa_2_rings_census")

# Create pandas dataframe from intersection table
rings_census_int_df = pd.DataFrame.spatial.from_table("rings_census_intersection")

# Subset intersection table to only the service area of interest
sa_2_int = rings_census_int_df[rings_census_int_df["Name"] == "5 - 30"].reset_index(drop = True)

In [119]:
# Calculate racial attributes
sa_2_census_df = interpolate_racial(sa_2_census_df, sa_2_int)

In [120]:
# Calculate education attributes
sa_2_edu = interpolate_edu(sa_2_census_df, sa_2_int, nhgis_edu)

[102]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [121]:
# Calculate income attributes
sa_2_inc = interpolate_inc(sa_2_census_df, sa_2_int, nhgis_inc)

[103]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [122]:
# Cast GEOID to int to facilitate joining with other census data
sa_2_census_df = sa_2_census_df.astype({"GEOID": "int64"})
sa_2_edu = sa_2_edu.astype({"GEOID": "int64"})
sa_2_inc = sa_2_inc.astype({"GEOID": "int64"})

# Join edited columns to the service area dataframe
sa_2_census_df = sa_2_census_df.merge(sa_2_edu, on = "GEOID", how = "left")
sa_2_census_df = sa_2_census_df.merge(sa_2_inc, on = "GEOID", how = "left")

In [123]:
# Write combined df back to feature class
sa_2_census_df.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "combined_census_sa_2_rings"), 
                                        sanitize_columns = False)

'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\combined_census_sa_2_rings'

## 30 - 90 mile service area

In [8]:
# Census tracts
# Set variables
in_features = "census_clipped"
clip_features = "service_area_3_rings"
out_feature_class = "sa_3_rings_census"

In [9]:
# Run clip
arcpy.analysis.Clip(in_features, clip_features, out_feature_class)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sa_3_rings_census'>

### Areal interpolation of census attributes

In [124]:
# Create pandas dataframe from census feature class
sa_3_census_df = pd.DataFrame.spatial.from_featureclass("sa_3_rings_census")

# Create pandas dataframe from intersection table
rings_census_int_df = pd.DataFrame.spatial.from_table("rings_census_intersection")

# Subset intersection table to only the service area of interest
sa_3_int = rings_census_int_df[rings_census_int_df["Name"] == "30 - 90"].reset_index(drop = True)

In [125]:
# Calculate racial attributes
sa_3_census_df = interpolate_racial(sa_3_census_df, sa_3_int)

In [126]:
# Calculate education attributes
sa_3_edu = interpolate_edu(sa_3_census_df, sa_3_int, nhgis_edu)

[102]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [127]:
# Calculate income attributes
sa_3_inc = interpolate_inc(sa_3_census_df, sa_3_int, nhgis_inc)

[103]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [128]:
# Cast GEOID to int to facilitate joining with other census data
sa_3_census_df = sa_3_census_df.astype({"GEOID": "int64"})
sa_3_edu = sa_3_edu.astype({"GEOID": "int64"})
sa_3_inc = sa_3_inc.astype({"GEOID": "int64"})

# Join edited columns to the service area dataframe
sa_3_census_df = sa_3_census_df.merge(sa_3_edu, on = "GEOID", how = "left")
sa_3_census_df = sa_3_census_df.merge(sa_3_inc, on = "GEOID", how = "left")

In [129]:
# Write combined df back to feature class
sa_3_census_df.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "combined_census_sa_3_rings"), 
                                        sanitize_columns = False)

'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\combined_census_sa_3_rings'

# Service Area with Disks

"Disks" means that the inner service areas are included in the outer service areas. For example, the 30 mile service area includes the 0-5 mile service area and the 5-30 mile service area.

## Split Network Dataset

In [166]:
# Create pandas dataframea from feature class
network_df = pd.DataFrame.spatial.from_featureclass("nd_output_disks")

In [172]:
# Split service area output into 3 separate polygons
sa_1 = network_df[network_df["ToBreak"] == 5.0]
sa_2 = network_df[network_df["ToBreak"] == 30.0]
sa_3 = network_df[network_df["ToBreak"] == 90.0]

In [174]:
# Write back to feature classes
sa_1.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "service_area_1_disks"), 
                                        sanitize_columns = False)
sa_2.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "service_area_2_disks"), 
                                        sanitize_columns = False)
sa_3.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "service_area_3_disks"), 
                                        sanitize_columns = False)

'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\service_area_3_disks'

## Calculate intersection of tracts in each service area

In [50]:
# Set variables
in_features = "census_clipped"
clip_features = "nd_output_disks"
out_feature_class = "nd_disks_census"

In [51]:
# Run clip
arcpy.analysis.Clip(in_features, clip_features, out_feature_class)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\nd_disks_census'>

In [52]:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/analysis/tabulate-intersection.htm
arcpy.analysis.TabulateIntersection(in_zone_features = "nd_disks_census", 
                                    zone_fields = "GEOID", # tract identifier
                                    in_class_features = "nd_output_disks", 
                                    out_table = "disks_census_intersection", 
                                    class_fields = "Name" # service area name
                                    )

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\disks_census_intersection'>

## 5 mile service area

In [175]:
# Set variables
in_features = "census_clipped"
clip_features = "service_area_1_disks"
out_feature_class = "sa_1_disks_census"

In [176]:
# Run clip
arcpy.analysis.Clip(in_features, clip_features, out_feature_class)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sa_1_disks_census'>

### Areal interpolation of census attributes

In [130]:
# Create pandas dataframe from census feature class
sa_1_census_df = pd.DataFrame.spatial.from_featureclass("sa_1_disks_census")

# Create pandas dataframe from intersection table
disks_census_int_df = pd.DataFrame.spatial.from_table("disks_census_intersection")

# Subset intersection table to only the service area of interest
sa_1_int = disks_census_int_df[disks_census_int_df["Name"] == "0 - 5"].reset_index(drop = True)

In [131]:
# Calculate racial attributes
sa_1_census_df = interpolate_racial(sa_1_census_df, sa_1_int)

In [132]:
# Calculate education attributes
sa_1_edu = interpolate_edu(sa_1_census_df, sa_1_int, nhgis_edu)

[102]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [133]:
# Calculate income attributes
sa_1_inc = interpolate_inc(sa_1_census_df, sa_1_int, nhgis_inc)

[103]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [134]:
# Cast GEOID to int to facilitate joining with other census data
sa_1_census_df = sa_1_census_df.astype({"GEOID": "int64"})
sa_1_edu = sa_1_edu.astype({"GEOID": "int64"})
sa_1_inc = sa_1_inc.astype({"GEOID": "int64"})

# Join edited columns to the service area dataframe
sa_1_census_df = sa_1_census_df.merge(sa_1_edu, on = "GEOID", how = "left")
sa_1_census_df = sa_1_census_df.merge(sa_1_inc, on = "GEOID", how = "left")

In [135]:
# Write combined df back to feature class
sa_1_census_df.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "combined_census_sa_1_disks"), 
                                        sanitize_columns = False)

'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\combined_census_sa_1_disks'

## 30 mile service area

In [177]:
# Set variables
in_features = "census_clipped"
clip_features = "service_area_2_disks"
out_feature_class = "sa_2_disks_census"

In [178]:
# Run clip
arcpy.analysis.Clip(in_features, clip_features, out_feature_class)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sa_2_disks_census'>

### Areal interpolation of census attributes

In [136]:
# Create pandas dataframe from census feature class
sa_2_census_df = pd.DataFrame.spatial.from_featureclass("sa_2_disks_census")

# Create pandas dataframe from intersection table
disks_census_int_df = pd.DataFrame.spatial.from_table("disks_census_intersection")

# Subset intersection table to only the service area of interest
sa_2_int = disks_census_int_df[disks_census_int_df["Name"] == "0 - 30"].reset_index(drop = True)

In [137]:
# Calculate racial attributes
sa_2_census_df = interpolate_racial(sa_2_census_df, sa_2_int)

In [138]:
# Calculate education attributes
sa_2_edu = interpolate_edu(sa_2_census_df, sa_2_int, nhgis_edu)

[102]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [139]:
# Calculate income attributes
sa_2_inc = interpolate_inc(sa_2_census_df, sa_2_int, nhgis_inc)

[103]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [140]:
# Cast GEOID to int to facilitate joining with other census data
sa_2_census_df = sa_2_census_df.astype({"GEOID": "int64"})
sa_2_edu = sa_2_edu.astype({"GEOID": "int64"})
sa_2_inc = sa_2_inc.astype({"GEOID": "int64"})

# Join edited columns to the service area dataframe
sa_2_census_df = sa_2_census_df.merge(sa_2_edu, on = "GEOID", how = "left")
sa_2_census_df = sa_2_census_df.merge(sa_2_inc, on = "GEOID", how = "left")

In [141]:
# Write combined df back to feature class
sa_2_census_df.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "combined_census_sa_2_disks"), 
                                        sanitize_columns = False)

'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\combined_census_sa_2_disks'

##  90 mile service area

In [155]:
# Set variables
in_features = "census_clipped"
clip_features = "service_area_3_disks"
out_feature_class = "sa_3_disks_census"

In [156]:
# Run clip
arcpy.analysis.Clip(in_features, clip_features, out_feature_class)

<Result 'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\sa_3_disks_census'>

### Areal interpolation of census attributes

In [142]:
# Create pandas dataframe from census feature class
sa_3_census_df = pd.DataFrame.spatial.from_featureclass("sa_3_disks_census")

# Create pandas dataframe from intersection table
disks_census_int_df = pd.DataFrame.spatial.from_table("disks_census_intersection")

# Subset intersection table to only the service area of interest
sa_3_int = disks_census_int_df[disks_census_int_df["Name"] == "0 - 90"].reset_index(drop = True)

In [143]:
# Calculate racial attributes
sa_3_census_df = interpolate_racial(sa_3_census_df, sa_3_int)

In [144]:
# Calculate education attributes
sa_3_edu = interpolate_edu(sa_3_census_df, sa_3_int, nhgis_edu)

[102]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [145]:
# Calculate income attributes
sa_3_inc = interpolate_inc(sa_3_census_df, sa_3_int, nhgis_inc)

[103]:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [146]:
# Cast GEOID to int to facilitate joining with other census data
sa_3_census_df = sa_3_census_df.astype({"GEOID": "int64"})
sa_3_edu = sa_3_edu.astype({"GEOID": "int64"})
sa_3_inc = sa_3_inc.astype({"GEOID": "int64"})

# Join edited columns to the service area dataframe
sa_3_census_df = sa_3_census_df.merge(sa_3_edu, on = "GEOID", how = "left")
sa_3_census_df = sa_3_census_df.merge(sa_3_inc, on = "GEOID", how = "left")

In [147]:
# Write combined df back to feature class
sa_3_census_df.copy().spatial.to_featureclass(location = os.path.join(gdb_path, "combined_census_sa_3_disks"), 
                                        sanitize_columns = False)

'C:\\Users\\kathr\\Documents\\outdoor-alliance\\san-gabriel\\san-gabriel-analysis\\sg_output.gdb\\combined_census_sa_3_disks'